In [1]:
import os
import pprint
import sys


path_cwd = os.getcwd()
print(path_cwd)
path_petsard = os.path.dirname(os.path.dirname(path_cwd))
print(path_petsard)
sys.path.append(path_petsard)

pp = pprint.PrettyPrinter(depth=2)

d:\Dropbox\89_other_application\GitHub\PETsARD\demo\dev
d:\Dropbox\89_other_application\GitHub\PETsARD


In [4]:
from PETsARD import (
    Loader,
    Splitter,
    Processor,
    Synthesizer,
    Describer,
)


load = Loader(
    filepath='benchmark://nist-national2018',
    na_values='N',
    column_types={
        'category': [
            'PUMA',
            'SEX',
            'MSP',
            'HISP',
            'RAC1P',
            'NOC',
            'NPF',
            'HOUSING_TYPE',
            'OWN_RENT',
            'INDP',
            'INDP_CAT',
            'EDU',
            'PINCP_DECILE',
            'DVET',
            'DREM',
            'DPHY',
            'DEYE',
            'DEAR',
            'PWGTP',
            'WGTP',
        ],
    },
)
load.load()
print(f"Raw data (Loader) # rn = {load.data.shape}")

split = Splitter(train_split_ratio=0.8, random_state=42)
split.split(data=load.data, metadata=load.metadata)
print(f"Split data (Splitter) # rn = {split.data[1]['train'].shape}")

proc = Processor(metadata=split.metadata)
proc.fit(data=split.data[1]['train'])
preproc_data = proc.transform(data=split.data[1]['train'])
print(f"Preproc data (Processor) # rn = {preproc_data.shape}")

syn = Synthesizer(
    method='smartnoise-aim',
    epsilon=0.3,
)
syn.create(data=preproc_data, metadata=proc._metadata)
syn.fit_sample()
print(f"Syn data (Synthesizer) by Preproc data w/ Preproc Metadata # rn = {syn.data_syn.shape}")

postproc_data = proc.inverse_transform(data=syn.data_syn)
print(f"Postproc data (Processor) # rn = {postproc_data.shape}")

Loader - Benchmarker: file benchmark\national2018.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
Raw data (Loader) # rn = (27111, 24)
Split data (Splitter) # rn = (21688, 24)
Preproc data (Processor) # rn = (100, 24)
Synthesizer (SmartNoise): Fitting mst.
Synthesizer (SmartNoise): Fitting mst spent 190.138 sec.
Synthesizer (SmartNoise): Sampling mst # 100 rows (same as input data) in 0.423 sec.
Syn data (Synthesizer) by Preproc data w/ Preproc Metadata # rn = (100, 24)
Postproc data (Processor) # rn = (100, 24)


d:\Dropbox\89_other_application\GitHub\PETsARD\.venv\lib\site-packages\snsynth\mst\mst.py:240: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[mask, col] = np.random.choice(extra, mask.sum())
d:\Dropbox\89_other_application\GitHub\PETsARD\.venv\lib\site-packages\snsynth\mst\mst.py:240: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[20 30 27 12 37 20  1 18 13 33  5  7  9 25 19 14 23 25 34  8 37  9  5 27
 15  4 29 24  4 19  5  0 33  8  5 21 30 23  5 30  3 19 33  6 13 18  4 12
  0 29  6 21 15  4  1 15  7 33 11 27 18 26 37 25 11 36 20  5 22  0 20  7
 28 29 23 11 20 23  3 25 29 14  5 12 36 33  9  2 16  5 37  4 33 15 20 36
  6 20 17 23]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[

In [20]:
syn = Synthesizer(
    method='smartnoise-aim',
    epsilon=0.3,
)
syn.create(data=preproc_data, metadata=proc._metadata)
syn.fit_sample(sample_num_rows=10000)
print(f"Syn data (Synthesizer) by Preproc data w/ Preproc Metadata # rn = {syn.data_syn.shape}")

Synthesizer (SmartNoise): Fitting aim.
24
Initial Sigma 380.0813430149839
Synthesizer (SmartNoise): Fitting aim spent 203.9575 sec.
Synthesizer (SmartNoise): Sampling aim # 10000 rows (same as manual input) in 0.1601 sec.
Syn data (Synthesizer) by Preproc data w/ Preproc Metadata # rn = (10000, 24)


In [5]:
syn.sample(sample_num_rows=5000)

Synthesizer (SmartNoise): Sampling mst # 5000 rows (same as manual input) in 0.2461 sec.


d:\Dropbox\89_other_application\GitHub\PETsARD\.venv\lib\site-packages\snsynth\mst\mst.py:240: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[mask, col] = np.random.choice(extra, mask.sum())
d:\Dropbox\89_other_application\GitHub\PETsARD\.venv\lib\site-packages\snsynth\mst\mst.py:240: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[21  4  1 ... 25 32 28]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[mask, col] = np.random.choice(extra, mask.sum())
d:\Dropbox\89_other_application\GitHub\PETsARD\.venv\lib\site-packages\snsynth\mst\mst.py:240: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[72 61 39

In [4]:
import yaml

from PETsARD import Executor


yaml_text: str = """---
Loader:
    national2018:
        filepath: 'benchmark://nist-national2018'
        na_values:
            'N'
        column_types:
            category:
                - PUMA
                - SEX
                - MSP
                - HISP
                - RAC1P
                - NOC
                - NPF
                - HOUSING_TYPE
                - OWN_RENT
                - INDP
                - INDP_CAT
                - EDU
                - PINCP_DECILE
                - DVET
                - DREM
                - DPHY
                - DEYE
                - DEAR
                - PWGTP
                - WGTP
Splitter:
    p0.8:
        train_split_ratio: 0.8
        random_state: 123
Preprocessor:
    encoder_label:
        encoder:
            PUMA: encoder_label
            SEX: encoder_label
            MSP: encoder_label
            HISP: encoder_label
            RAC1P: encoder_label
            NOC: encoder_label
            NPF: encoder_label
            HOUSING_TYPE: encoder_label
            OWN_RENT: encoder_label
            INDP: encoder_label
            INDP_CAT: encoder_label
            EDU: encoder_label
            PINCP_DECILE: encoder_label
            DVET: encoder_label
            DREM: encoder_label
            DPHY: encoder_label
            DEYE: encoder_label
            DEAR: encoder_label
            PWGTP: encoder_label
            WGTP: encoder_label
        sequence:
            - 'missing'
            - 'outlier'
            - 'encoder'
            - 'scaler'
Synthesizer:
    default:
        method: default
        sample_num_rows: 5000

Postprocessor:
    default:
        method: 'default'
..."""

# pp.pprint(yaml_text)

cfg = yaml.safe_load(yaml_text)
pp.pprint(cfg)

yaml_file: str = f'temp_eval.yaml'
with open(yaml_file, 'w') as f:
    f.write(yaml_text)

exec = Executor(config=yaml_file)
exec.run()

os.remove(yaml_file)

print(exec.status.status['Synthesizer']['operator'].get_result().shape)

{'Loader': {'national2018': {...}},
 'Postprocessor': {'default': {...}},
 'Preprocessor': {'encoder_label': {...}},
 'Splitter': {'p0.8': {...}},
 'Synthesizer': {'default': {...}}}
Now is Loader with national2018...
Loader - Benchmarker: file benchmark\national2018.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
Now is Splitter with p0.8_[1-1]...
Now is Preprocessor with encoder_label...
Now is Synthesizer with default...
yaaaa
{'row_num': 27111, 'col_num': 24, 'na_percentage': 0.995094, 'row_num_after_split': {'train': 21688, 'validation': 5423}, 'row_num_after_preproc': 104}
Synthesizer (SDV): Fitting GaussianCopula.
Synthesizer (SDV): Fitting GaussianCopula spent 1.7985 sec.
Synthesizer (SDV): Sampling GaussianCopula # 5000 rows (same as manual input) in 0.9339 sec.
Now is Postprocessor with default...
(5000, 24)


In [38]:
import yaml

from PETsARD import Executor


yaml_text: str = """---
Loader:
    national2018:
        filepath: 'benchmark://nist-national2018'
        na_values:
            'N'
        column_types:
            category:
                - PUMA
                - SEX
                - MSP
                - HISP
                - RAC1P
                - NOC
                - NPF
                - HOUSING_TYPE
                - OWN_RENT
                - INDP
                - INDP_CAT
                - EDU
                - PINCP_DECILE
                - DVET
                - DREM
                - DPHY
                - DEYE
                - DEAR
                - PWGTP
                - WGTP
Splitter:
    p0.8:
        train_split_ratio: 0.8
        random_state: 123
Preprocessor:
    encoder_label:
        encoder:
            PUMA: encoder_label
            SEX: encoder_label
            MSP: encoder_label
            HISP: encoder_label
            RAC1P: encoder_label
            NOC: encoder_label
            NPF: encoder_label
            HOUSING_TYPE: encoder_label
            OWN_RENT: encoder_label
            INDP: encoder_label
            INDP_CAT: encoder_label
            EDU: encoder_label
            PINCP_DECILE: encoder_label
            DVET: encoder_label
            DREM: encoder_label
            DPHY: encoder_label
            DEYE: encoder_label
            DEAR: encoder_label
            PWGTP: encoder_label
            WGTP: encoder_label
        sequence:
            - 'missing'
            - 'outlier'
            - 'encoder'
            - 'scaler'
Synthesizer:
    aim_5.0:
        method: 'smartnoise-aim'
        epsilon: 0.3
        sample_num_rows: 5000

Postprocessor:
    default:
        method: 'default'
..."""

# pp.pprint(yaml_text)

cfg = yaml.safe_load(yaml_text)
pp.pprint(cfg)

yaml_file: str = f'temp_eval.yaml'
with open(yaml_file, 'w') as f:
    f.write(yaml_text)

exec = Executor(config=yaml_file)
exec.run()

os.remove(yaml_file)

print(exec.status.status['Synthesizer']['operator'].get_result().shape)

{'Loader': {'national2018': {...}},
 'Postprocessor': {'default': {...}},
 'Preprocessor': {'encoder_label': {...}},
 'Splitter': {'p0.8': {...}},
 'Synthesizer': {'aim_5.0': {...}}}
Now is Loader with national2018...
Loader - Benchmarker: file benchmark\national2018.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
Now is Splitter with p0.8_[1-1]...
Now is Preprocessor with encoder_label...
Now is Synthesizer with aim_5.0...
Synthesizer (SmartNoise): Fitting aim.
24
Initial Sigma 380.0813430149839
Synthesizer (SmartNoise): Fitting aim spent 295.2838 sec.
Synthesizer (SmartNoise): Sampling aim # 5000 rows (same as manual input) in 0.4939 sec.
Now is Postprocessor with default...
(5000, 24)


In [2]:
import yaml

from PETsARD import Executor


yaml_text: str = """---
Loader:
    national2018:
        filepath: 'benchmark://nist-national2018'
        na_values:
            'N'
        column_types:
            category:
                - PUMA
                - SEX
                - MSP
                - HISP
                - RAC1P
                - NOC
                - NPF
                - HOUSING_TYPE
                - OWN_RENT
                - INDP
                - INDP_CAT
                - EDU
                - PINCP_DECILE
                - DVET
                - DREM
                - DPHY
                - DEYE
                - DEAR
                - PWGTP
                - WGTP
Splitter:
    p0.8:
        train_split_ratio: 0.8
        random_state: 123
Preprocessor:
    encoder_label:
        encoder:
            PUMA: encoder_label
            SEX: encoder_label
            MSP: encoder_label
            HISP: encoder_label
            RAC1P: encoder_label
            NOC: encoder_label
            NPF: encoder_label
            HOUSING_TYPE: encoder_label
            OWN_RENT: encoder_label
            INDP: encoder_label
            INDP_CAT: encoder_label
            EDU: encoder_label
            PINCP_DECILE: encoder_label
            DVET: encoder_label
            DREM: encoder_label
            DPHY: encoder_label
            DEYE: encoder_label
            DEAR: encoder_label
            PWGTP: encoder_label
            WGTP: encoder_label
        sequence:
            - 'missing'
            - 'outlier'
            - 'encoder'
            - 'scaler'
Synthesizer:
    aim_5.0:
        method: 'smartnoise-aim'
        epsilon: 0.3

Postprocessor:
    default:
        method: 'default'
..."""

# pp.pprint(yaml_text)

cfg = yaml.safe_load(yaml_text)
pp.pprint(cfg)

yaml_file: str = f'temp_eval.yaml'
with open(yaml_file, 'w') as f:
    f.write(yaml_text)

exec = Executor(config=yaml_file)
exec.run()

os.remove(yaml_file)

print(exec.status.status['Synthesizer']['operator'].get_result().shape)

{'Loader': {'national2018': {...}},
 'Postprocessor': {'default': {...}},
 'Preprocessor': {'encoder_label': {...}},
 'Splitter': {'p0.8': {...}},
 'Synthesizer': {'aim_5.0': {...}}}
Now is Loader with national2018...
Loader - Benchmarker: file benchmark\national2018.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
Now is Splitter with p0.8_[1-1]...
Now is Preprocessor with encoder_label...
Now is Synthesizer with aim_5.0...
yaaaa
{'row_num': 27111, 'col_num': 24, 'na_percentage': 0.995094, 'row_num_after_split': {'train': 21688, 'validation': 5423}, 'row_num_after_preproc': 104}
yaaaa
{'row_num': 27111, 'col_num': 24, 'na_percentage': 0.995094, 'row_num_after_split': {'train': 21688, 'validation': 5423}, 'row_num_after_preproc': 104}
Synthesizer (SmartNoise): Fitting aim.
24
Initial Sigma 380.0813430149839
Synthesizer (SmartNoise): Fitting aim spent 232.6141 sec.
Synthesizer (SmartNoise): Sampling aim # 21688 rows (sa

In [3]:
print(exec.status.status['Loader']['operator'].get_result().shape)
print(exec.status.status['Splitter']['operator'].get_result()['train'].shape)
print(exec.status.status['Preprocessor']['operator'].get_result().shape)
print(exec.status.status['Synthesizer']['operator'].get_result().shape)

(27111, 24)
(21688, 24)
(104, 24)
(21688, 24)


In [46]:
exec.status.status['Synthesizer']['operator'].synthesizer.config['metadata'].metadata

{'col': {'PUMA': {'dtype': CategoricalDtype(categories=['01-01301', '06-07502', '06-08507', '08-00803', '13-04600',
                     '17-03529', '17-03531', '19-01700', '24-01004', '26-02702',
                     '28-01100', '29-01901', '30-00600', '32-00405', '36-03710',
                     '36-04010', '38-00100', '40-00200', '51-01301', '51-51255'],
   , ordered=False, categories_dtype=object),
   'na_percentage': 0.0,
   'infer_dtype': 'categorical'},
  'AGEP': {'dtype': dtype('int8'),
   'na_percentage': 0.0,
   'infer_dtype': 'numerical'},
  'SEX': {'dtype': CategoricalDtype(categories=['1', '2'], ordered=False, categories_dtype=object),
   'na_percentage': 0.0,
   'infer_dtype': 'categorical'},
  'MSP': {'dtype': CategoricalDtype(categories=['1', '2', '3', '4', '5', '6'], ordered=False, categories_dtype=object),
   'na_percentage': 0.159972,
   'infer_dtype': 'categorical'},
  'HISP': {'dtype': CategoricalDtype(categories=['0', '1', '2', '3', '4'], ordered=False, categories

In [42]:
exec.status.status['Synthesizer']['operator'].synthesizer.synthesizer.sample_num_rows_as

'input data'